# Camada Silver – Tratamento e Padronização

## Objetivo
Transformar os dados da camada Bronze em uma base canônica (Silver), com:

- padronização de tipos (ex.: datas)
- normalização de atributos (ex.: duração em minutos / temporadas)
- normalização relacional de campos multivalorados (N:N), garantindo melhor qualidade analítica

## Fonte
- Entrada: `bronze.disney_titles_raw`
- Saídas:
  - `silver.disney_titles_clean` (tabela canônica)
  - tabelas normalizadas:
    - `silver.title_genres`
    - `silver.title_countries`
    - `silver.title_directors`
    - `silver.title_cast`


## Principais Transformações

- Conversão de `date_added` para tipo data
- Normalização de duração (minutos / temporadas)
- Padronização de campos categóricos


In [0]:
from pyspark.sql import functions as F

df = spark.table("bronze.disney_titles_raw")

print("1. Iniciando transformações de campos com padronizações leves")
# Padronizações leves e canônicas
df_silver = (
    df
    # trims básicos
    .withColumn("show_id", F.trim(F.col("show_id")))
    .withColumn("type", F.trim(F.col("type")))
    .withColumn("title", F.trim(F.col("title")))
    .withColumn("rating", F.trim(F.col("rating")))
    .withColumn("listed_in", F.trim(F.col("listed_in")))
    .withColumn("country", F.trim(F.col("country")))
    .withColumn("director", F.trim(F.col("director")))
    .withColumn("cast", F.trim(F.col("cast")))
    .withColumn("description", F.trim(F.col("description")))
    
    # parse de data
    #.withColumn("date_added_dt", F.to_date("date_added", "yyyy-MM-dd"))
    .withColumn("date_added_dt", F.expr("try_to_date(date_added)"))
    
    # normalização de duração (num/unidade)
    .withColumn("duration_num", F.regexp_extract("duration", r"(\d+)", 1).cast("int"))
    .withColumn(
        "duration_unit",
        F.when(F.col("duration").contains("min"), F.lit("min"))
         .when(F.col("duration").contains("Season"), F.lit("season"))
         .otherwise(F.lit(None))
    )
    .withColumn("duration_minutes", F.when(F.col("duration_unit") == "min", F.col("duration_num")))
    .withColumn("seasons", F.when(F.col("duration_unit") == "season", F.col("duration_num")))
    
    # tipagem do ano de lançamento
    .withColumn("release_year_int", F.col("release_year").cast("int"))
    
    # normalização de strings vazias para null (boa prática)
    .withColumn("director", F.nullif(F.col("director"), F.lit("")))
    .withColumn("cast", F.nullif(F.col("cast"), F.lit("")))
    .withColumn("country", F.nullif(F.col("country"), F.lit("")))
    .withColumn("rating", F.nullif(F.col("rating"), F.lit("")))
)

print("Transformações de campos com padronizações leves concluídas (Remoção de vazios, normalização de duração, conversão de data, tipo do ano de lançamento e normalização de strings vazias para null).")

# Remover duplicidades por show_id (se existirem) mantendo a primeira ocorrência
print("2. Removendo duplicadas no campo show_id")
df_silver = df_silver.dropDuplicates(["show_id"])
print("Duplicadas no campo show_id removidas.")

# Persistência canônica Silver
print("3. Persistindo dados canônicos camada Silver em tabela Delta")
(df_silver
 .write
 .format("delta")
 .mode("overwrite")
 .saveAsTable("silver.disney_titles_clean"))
print("Tabela Delta salva")

print("4. Exibindo 5 primeiras linhas da tabela Delta - Camada Silver")
display(df_silver.limit(5))

1. Iniciando transformações de campos com padronizações leves
Transformações de campos com padronizações leves concluídas (Remoção de vazios, normalização de duração, conversão de data, tipo do ano de lançamento e normalização de strings vazias para null).
2. Removendo duplicadas no campo show_id
Duplicadas no campo show_id removidas.
3. Persistindo dados canônicos camada Silver em tabela Delta
Tabela Delta salva
4. Exibindo 5 primeiras linhas da tabela Delta - Camada Silver


show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,date_added_dt,duration_num,duration_unit,duration_minutes,seasons,release_year_int
s537,TV Show,Disney Prop Culture,null,Daniel Lanigan,United States,"May 1, 2020",2020,TV-PG,1 Season,Docuseries,"Collector Dan Lanigan reunites iconic Disney movie props with filmmakers, actors, and crew.",null,1,season,null,1,2020
s722,Movie,Alice in Wonderland,"Clyde Geronimi, Hamilton Luske, Wilfred Jackson","Ed Wynn, Richard Haydn, Sterling Holloway, Jerry Colonna, Verna Felton, Pat O'Malley",United States,"November 12, 2019",1951,G,79 min,"Action-Adventure, Animation, Family",Join Alice as she chases the White Rabbit into a topsy-turvy world.,null,79,min,79,null,1951
s47,Movie,Santa Buddies: The Legend of Santa Paws,Robert Vince,"Christopher Lloyd, Danny Woodburn, Craig Anton, Michael Teigen, Ben Giroux, Paul Rae","United States, Canada","November 5, 2021",2009,G,93 min,"Comedy, Fantasy, Kids","The Buddies remind Santa Paws' feisty son, Puppy Paws, about the true meaning of Christmas.",null,93,min,93,null,2009
s266,TV Show,Star Wars Vintage: Clone Wars 2D Micro-Series,null,"Mat Lucas, James Arnold Taylor, Tom Kane, Grey DeLisle, Corey Burton, André Sogliuzzo",null,"April 2, 2021",2021,TV-Y7-FV,1 Season,Series,"Enjoy this thrilling, Emmy® Award-winning animated series that continues the Star Wars story.",null,1,season,null,1,2021
s715,Movie,A Goofy Movie,Kevin Lima,"Bill Farmer, Jason Marsden, Jim Cummings, Kellie Martin, Rob Paulsen, Wallace Shawn","United States, Australia, France, Canada","November 12, 2019",1995,G,81 min,"Action-Adventure, Animation, Comedy",Goofy and Max hit the road and get up to their floppy ears in misadventure.,null,81,min,81,null,1995


## Validação da quantidade de linhas da camada Silver persistida
Demonstrando que a mesma quantidade de linhas foi mantida

In [0]:
%sql
SELECT COUNT (*) FROM silver.disney_titles_clean

COUNT(*)
1450


## Normalização de Campos Multivalorados (N:N)

O dataset contém campos com múltiplos valores em uma única coluna (separados por vírgula),
o que prejudica análises e modelagem estrela.

Nesta etapa, normalizamos para tabelas auxiliares do tipo:
- show_id + valor_normalizado

Campos normalizados:
- `listed_in` → gêneros (title_genres)
- `country` → países (title_countries)
- `director` → diretores (title_directors)
- `cast` → elenco (title_cast)


In [0]:
from pyspark.sql import DataFrame

print("5. Normalizando campos multivalorados")

def normalize_multivalue(df_base: DataFrame, id_col: str, src_col: str, out_col: str, out_table: str):
    """
    Cria uma tabela normalizada (N:N) a partir de uma coluna multivalorada separada por vírgula.
    - Remove espaços extras
    - Remove valores vazios
    - Remove duplicidades show_id + valor
    """
    normalized = (
        df_base
        .select(F.col(id_col).alias(id_col), F.col(src_col).alias(src_col))
        .filter(F.col(src_col).isNotNull())
        .withColumn(out_col, F.explode(F.split(F.col(src_col), r",\s*")))
        .withColumn(out_col, F.trim(F.col(out_col)))
        .filter(F.col(out_col).isNotNull() & (F.col(out_col) != ""))
        .select(id_col, out_col)
        .dropDuplicates([id_col, out_col])
    )
    
    (normalized
     .write
     .format("delta")
     .mode("overwrite")
     .saveAsTable(out_table))
    
    return normalized



5. Normalizando campos multivalorados


In [0]:
base = spark.table("silver.disney_titles_clean")

print("- Normalização do campo 'listed_in' (gêneros)")
# Gêneros: listed_in
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="listed_in",
    out_col="genre",
    out_table="silver.title_genres"
)

print("- Normalização do campo 'country'")
# Países: country
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="country",
    out_col="country_name",
    out_table="silver.title_countries"
)

print("- Normalização do campo 'director'")
# Diretores: director
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="director",
    out_col="director_name",
    out_table="silver.title_directors"
)

print("- Normalização do campo 'cast'")
# Elenco: cast
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="cast",
    out_col="cast_name",
    out_table="silver.title_cast"
)
print("Normalização concluída.")

- Normalização do campo 'listed_in' (gêneros)
- Normalização do campo 'country'
- Normalização do campo 'director'
- Normalização do campo 'cast'
Normalização concluída.


## Validações de Qualidade (Silver)

Objetivos:
- confirmar que a Silver canônica foi criada
- medir completude (nulos) por atributo relevante
- confirmar volume das tabelas normalizadas
- identificar possíveis inconsistências em domínios (ex.: type, rating)


In [0]:
%sql
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN show_id IS NULL THEN 1 ELSE 0 END) AS null_show_id,
  SUM(CASE WHEN type IS NULL THEN 1 ELSE 0 END) AS null_type,
  SUM(CASE WHEN title IS NULL THEN 1 ELSE 0 END) AS null_title,
  SUM(CASE WHEN listed_in IS NULL THEN 1 ELSE 0 END) AS null_listed_in,
  SUM(CASE WHEN release_year_int IS NULL THEN 1 ELSE 0 END) AS null_release_year,
  SUM(CASE WHEN date_added_dt IS NULL THEN 1 ELSE 0 END) AS null_date_added
FROM silver.disney_titles_clean;


total,null_show_id,null_type,null_title,null_listed_in,null_release_year,null_date_added
1450,0,0,0,0,0,3


In [0]:
%sql
SELECT type, COUNT(*) AS qtd
FROM silver.disney_titles_clean
GROUP BY type
ORDER BY qtd DESC;


type,qtd
Movie,1052
TV Show,398


In [0]:
%sql
SELECT 'title_genres' AS tabela, COUNT(*) AS linhas FROM silver.title_genres
UNION ALL
SELECT 'title_countries', COUNT(*) FROM silver.title_countries
UNION ALL
SELECT 'title_directors', COUNT(*) FROM silver.title_directors
UNION ALL
SELECT 'title_cast', COUNT(*) FROM silver.title_cast;


tabela,linhas
title_genres,3910
title_countries,1528
title_directors,1121
title_cast,5899


## Evidência e Rastreabilidade

As transformações desta camada (tipagem, normalização de duração e normalização relacional)
são reproduzíveis a partir deste notebook versionado no GitHub integrado ao Databricks.

As tabelas geradas na Silver serão usadas na Gold para:
- modelo estrela (dimensões + fato)
- análises de diversidade e perfis
- agregações e métricas de negócio


Observações importantes:

- Não normalizei `description` porque não é multivalorado e normalmente é usado em NLP/EDA (Gold/ML).
- Não normalizei `rating` porque é categórico simples e já está canônico na Silver.
- Mantive as tabelas normalizadas na camada Silver porque facilitam criar dimensões e pontes na Gold e evitam repetição de parsing/explode em toda query analítica


In [0]:
%sql

SELECT * FROM silver.disney_titles_clean;

show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,date_added_dt,duration_num,duration_unit,duration_minutes,seasons,release_year_int
s537,TV Show,Disney Prop Culture,null,Daniel Lanigan,United States,"May 1, 2020",2020,TV-PG,1 Season,Docuseries,"Collector Dan Lanigan reunites iconic Disney movie props with filmmakers, actors, and crew.",2020-05-01,1,season,null,1,2020
s722,Movie,Alice in Wonderland,"Clyde Geronimi, Hamilton Luske, Wilfred Jackson","Ed Wynn, Richard Haydn, Sterling Holloway, Jerry Colonna, Verna Felton, Pat O'Malley",United States,"November 12, 2019",1951,G,79 min,"Action-Adventure, Animation, Family",Join Alice as she chases the White Rabbit into a topsy-turvy world.,2019-11-12,79,min,79,null,1951
s47,Movie,Santa Buddies: The Legend of Santa Paws,Robert Vince,"Christopher Lloyd, Danny Woodburn, Craig Anton, Michael Teigen, Ben Giroux, Paul Rae","United States, Canada","November 5, 2021",2009,G,93 min,"Comedy, Fantasy, Kids","The Buddies remind Santa Paws' feisty son, Puppy Paws, about the true meaning of Christmas.",2021-11-05,93,min,93,null,2009
s266,TV Show,Star Wars Vintage: Clone Wars 2D Micro-Series,null,"Mat Lucas, James Arnold Taylor, Tom Kane, Grey DeLisle, Corey Burton, André Sogliuzzo",null,"April 2, 2021",2021,TV-Y7-FV,1 Season,Series,"Enjoy this thrilling, Emmy® Award-winning animated series that continues the Star Wars story.",2021-04-02,1,season,null,1,2021
s715,Movie,A Goofy Movie,Kevin Lima,"Bill Farmer, Jason Marsden, Jim Cummings, Kellie Martin, Rob Paulsen, Wallace Shawn","United States, Australia, France, Canada","November 12, 2019",1995,G,81 min,"Action-Adventure, Animation, Comedy",Goofy and Max hit the road and get up to their floppy ears in misadventure.,2019-11-12,81,min,81,null,1995
s534,TV Show,Birth of Europe,null,Mark Halliley,United Kingdom,"May 1, 2020",2014,TV-G,1 Season,Docuseries,Birth of Europe reveals the extraordinary story of how Europe has been created by Nature’s forces.,2020-05-01,1,season,null,1,2014
s619,TV Show,Star Wars: The Clone Wars,null,"Tom Kane, Matt Lanter, Ashley Eckstein, James Arnold Taylor, Dee Baker, Corey Burton",United States,"February 21, 2020",2008,TV-PG,7 Seasons,"Action-Adventure, Animation, Science Fiction","The saga continues, set in a faraway galaxy!",2020-02-21,7,season,null,7,2008
s678,Movie,George of the Jungle 2,David Grossman,"Thomas Church, Julie Benz, Christina Pickles, Angus Jones, Kelly Miller, John Kassir","United States, Australia","December 20, 2019",2003,PG,89 min,"Action-Adventure, Comedy, Family","George must rescue his family, then save the jungle from becoming mulch.",2019-12-20,89,min,89,null,2003
s250,Movie,Treasure Buddies,Robert Vince,"Richard Riehle, Mason Cook, Adam Alexi-Malle, Lochlyn Munro, Mo Gallini, Christopher Maleki","United States, Canada","April 16, 2021",2012,G,96 min,"Action-Adventure, Comedy, Kids",Travel across the globe on a treasure hunt with the adorable thrill-seeking Buddies.,2021-04-16,96,min,96,null,2012
s49,Movie,Space Buddies,Robert Vince,"F.T. Anderson, Diedrich Bader, Field Cate, Kendall Cross, Mike Dopud, Mark Downey","United States, Canada","November 5, 2021",2009,G,87 min,"Action-Adventure, Comedy, Kids","Disney's most adventurous talking pups take one small step for dog, one giant leap for dogkind.",2021-11-05,87,min,87,null,2009
